# picture跑模型数据生成

picture的特征抽取

In [9]:
## Feature 1:HSV

# pip install opencv-python-headless
import cv2
import numpy as np


def calculate_average_hsv(image_path):
    # Load the image
    image = cv2.imread(image_path)
    # Convert the image from BGR to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Calculate the average hue, saturation, and value
    average_hue = np.mean(hsv_image[:, :, 0])
    average_saturation = np.mean(hsv_image[:, :, 1])
    average_value = np.mean(hsv_image[:, :, 2])
    return average_hue, average_saturation, average_value

# # 对 h s v进行大标签
# def categorize_image_by_average_hue(average_hue):
#     # Adjusting for the scale wrapping of red
#     if (0 <= average_hue <= 34) or (170 <= average_hue <= 179):
#         return 'Warm Dominant'
#     else:
#         return 'Cool Dominant'


# def classify_saturation(average_saturation):
#     saturation_category = "Low" if average_saturation < 64 else "High" if average_saturation > 191 else "Medium"
#     return saturation_category


# def classify_saturation(average_value):
#     value_category = "Low" if average_value < 64 else "High" if average_value > 191 else "Medium"
#     return value_category


# # Example usage
# image_path = '1.png'
# average_hue, average_saturation, average_value = calculate_average_hsv(image_path)
# print([average_hue, average_saturation, average_value])

# h_cate = categorize_image_by_average_hue(average_hue)
# s_cate = classify_saturation(average_saturation)
# v_cate = classify_saturation(average_value)
# print(h_cate,s_cate,v_cate)


In [10]:
## Feature 2: smilling face detect 

import cv2

def detect_smiling_faces(image_path):
    # Load the Haar Cascade classifiers for face and smile detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')
    
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    smiling_faces_count = 0
    
    # For each detected face, look for a smile
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        smiles = smile_cascade.detectMultiScale(face_roi, scaleFactor=1.8, minNeighbors=20, minSize=(25, 25), flags=cv2.CASCADE_SCALE_IMAGE)
        
        # If at least one smile is detected in the face ROI, count the face as smiling
        if len(smiles) > 0:
            smiling_faces_count += 1
            
    return smiling_faces_count

# # Example usage
# image_path = '3.png'
# smiling_faces = detect_smiling_faces(image_path)
# print(f"Number of smiling faces detected: {smiling_faces}")


In [11]:
## Feature 3: Clarity

# pip install opencv-python-headless

import cv2

def quantify_image_quality_and_clarity(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply the Laplacian operator to the image
    laplacian_var = cv2.Laplacian(image, cv2.CV_64F).var()
    
    return laplacian_var

# # Example usage
# image_path = '1.png'
# sharpness_measure = quantify_image_quality_and_clarity(image_path)
# print(f"Sharpness (Variance of Laplacian): {sharpness_measure}")

In [7]:
## Feature 4: Textual Details 图片识别文本
### 2 outcome variable (text; positive&negative)


# pip install pytesseract pillow
# import pytesseract

# https://blog.csdn.net/yuan2019035055/article/details/129754864

# from PIL import Image
# import pytesseract

# def detect_text(image_path):
#     # Load the image
#     image = Image.open(image_path)
    
#     # Use Pytesseract to detect text
#     text = pytesseract.image_to_string(image)
    
#     return text.strip()

# # Example usage
# image_path = '1.png'
# detected_text = detect_text(image_path)
# print(detected_text)
# if detected_text:
#     print(f"Detected text: {detected_text}")
# else:
#     print("No text detected in the image.")

In [ ]:
## Feature 5: color diversity
### 1 outcome variable (uniqueness_score)


import cv2
import numpy as np

def quantify_color_uniqueness(image_path):
    # Load the image
    image = cv2.imread(image_path)
    # Convert to HSV for more meaningful color analysis
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Flatten the image array and calculate unique colors
    unique_colors = np.unique(hsv_image.reshape(-1, hsv_image.shape[2]), axis=0)
    # The number of unique colors can serve as a basic proxy for uniqueness
    color_uniqueness_score = len(unique_colors)
    return color_uniqueness_score

# # Example usage
# image_path = '1.png'
# uniqueness_score = quantify_color_uniqueness(image_path)
# print(f"Color Uniqueness Score: {uniqueness_score}")

In [ ]:
## Feature 6: object detection ===> yolo11  tf2(Python3.8.0)的环境可以跑   图片识别物体
### 1 outcome variable (uniqueness_score)

from ultralytics import YOLO
import os
model = YOLO('data/pic/yolo11n.pt')  # load a pretrained YOLO detection model


def detect_pic_object(path):
  try:
    res = []
    results = model(path)  # predict on an image
    # 遍历每个检测结果
    for i, result in enumerate(results):
        # 打印图像路径
        print(f"Image {i + 1}: {result.path}")
        # 获取检测框、类别和置信度
        boxes = result.boxes  # Boxes object for bbox outputs
        masks = result.masks  # Masks object for segmentation masks outputs (if applicable)
        probs = result.probs  # Class probabilities (classification only)
        # 打印每个检测框的信息
        for box in boxes:
            cls = box.cls  # 类别ID
            # 假设你有一个类别名称列表
            class_names = model.names  # 获取模型的类别名称映射
            class_name = class_names[int(cls)]  # 将类别ID转换为类别名称
            # 打印检测结果
            # print(class_name)
            res.append(class_name)
  except:
     res = ''
  return res


# fs = open('data/pic/result_object_detect_yolo11.txt','a+',encoding='utf-8')
# count = 0
# # for item in os.listdir("pic_data")[:]:
# for item in os.listdir("photos")[:]:
#     count+=1
#     print(count,item)
#     all_path = "photos"+'//'+item
#     res = detect_pic_object(all_path)
#     print([item.split('.')[0]+"&"+str(res)], file=fs)



In [ ]:
# memory score

from resmem import ResMem, transformer
model = ResMem(pretrained=True)

def pic_score_pre(pic_path):
    img = Image.open(pic_path) # This loads your image into memory
    img = img.convert('RGB') 
    # This will convert your image into RGB, for instance if it's a PNG (RGBA) or if it's black and white.
    model.eval()
    # Set the model to inference mode.
    image_x = transformer(img)
    # Run the preprocessing function
    # print(image_x.shape)
    prediction = model(image_x.view(-1, 3, 227, 227))
    # For a single image, the image must be reshaped into a batch
    # with size 1.
    # Get your prediction!
    return round(prediction[0][0].item(),3)


In [ ]:
# import pandas as pd

# fs = open('result.txt','a+',encoding='utf-8')
# path =r'\yelp_memory\data\pic'

# count = 0
# result = []
# for pic in os.listdir(path)[:]:
#     count+=1
#     print(count,pic.split('.')[0])
#     # print(count,path+'\\'+pic,pic.split('.')[0])
#     image_path = path+'\\'+pic
#     average_hue, average_saturation, average_value = calculate_average_hsv(image_path)
#     # print([average_hue, average_saturation, average_value])
#     number_face,smiling_faces_count = detect_smiling_faces(image_path)
#     # print(smiling_faces_count)
#     sharpness_measure = quantify_image_quality_and_clarity(image_path)
#     # print(sharpness_measure)
#     detected_text = detect_text(image_path)
#     # print(detected_text)
#     uniqueness_score = quantify_color_uniqueness(image_path)
#     # print(uniqueness_score)
#     object_detection= detect_pic_object(image_path)
#     # print(object_detection)

#     print([pic.split('.')[0],
#                   average_hue, average_saturation, average_value,
#                   number_face,
#                   smiling_faces_count,
#                   sharpness_measure,
#                   detected_text,
#                   uniqueness_score,
#                  # object_detection
# ],file=fs)
    
#     result.append([pic.split('.')[0],
#                   average_hue, average_saturation, average_value,
#                   number_face,
#                   smiling_faces_count,
#                   sharpness_measure,
#                   detected_text,
#                   uniqueness_score,
#                #   object_detection
#                  ]
#                   )

# result_df = pd.DataFrame(result,columns=['pic_filename',
#                   'average_hue', 'average_saturation', 'average_value',
#                   'number_face','smiling_faces_count','sharpness_measure','detected_text','uniqueness_score'
#                  # ,"object_detection"
#])

# result_df.to_excel('generate_result.xlsx',index=False)

In [ ]:
# 处理yolo的数据 生成3个变量

import pandas as pd

person_result = []

with open("data/pic/result_object_detect_yolo11.txt") as f:
    lines = f.readlines()
    for l in lines[:]:
        name,detect_content = l.split("&")
        pic_name = name.replace('["',"").replace("['","")
        pic_detect_content = str(detect_content).replace('[',"").replace(']',"").replace("'","").replace('"','')
        person_count = detect_content.count("person")
        # person_judge = str(detect_content).find("person")
        if person_count==0:
            person_exist = 0
        else:
            person_exist = 1

        # print(pic_name,
        #       person_count,
        #       person_exist,
        #       pic_detect_content
        #       )
        person_result.append([pic_name,str(person_count),str(person_exist),pic_detect_content])

pd.DataFrame(person_result,columns=['photo_id','person_count','person_exist','objects_content']).to_excel("data/pic/person_result.xlsx",index=False)
print("数据保存成功！")

数据保存成功！


In [8]:
# # 图片构图位置 和图片美学（如果需要执行这个代码，需要把这段放在数据导入的下面）
# df3_position = pd.read_excel('data/pic/图像位置构图结果.xlsx',header=None)
# df3_position[0]=df3_position[0].str.replace('"',"")
# df3_position[0]=df3_position[0].str.replace('.jpg',"")
# # df3_position.head()
# df4_beauty = pd.read_csv('data/pic/图片美学.csv',header=None)
# df4_beauty[0]=df4_beauty[0].str.replace('.jpg',"")
# # df4_beauty.head()

# pic1 = pd.merge(pic,df3_position,left_on='pic_filename',right_on=0,how='left')
# pic2 = pd.merge(pic1,df4_beauty,left_on='pic_filename',right_on=0,how='left')
# pic2.rename(columns={'1_x': 'pic_position','1_y':"pic_beauty"}, inplace=True)
# pic2 = pic2.drop(['0_x', '0_y'], axis=1)


In [9]:
# 数据格式的处理函数
def protect_excel_formula(df):
    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: f"'{x}" if isinstance(x, str) and x.startswith(('=', '+', '-', '@')) else x
        )
    return df

In [ ]:
# 对yolo11的结果进行处理并合并到generate_result数据上
import pandas as pd

business = pd.read_excel("data/excel/yelp_academic_dataset_business.xlsx")
# 遍历图片路径生成（特征变量）最终生成 generate_result数据集
pic = pd.read_excel("data/pic/generate_result.xlsx")
# yolo11的检测结果处理后的变量
person_result = pd.read_excel("data/pic/person_result.xlsx")
pic_bus = pd.read_excel("data/excel/photos_result.xlsx")
# 对于为什么要过滤掉0的数据，因为部分图片是坏掉的，所以memory score是0
pic_bus = pic_bus[pic_bus['memory_score']!=0]
print(len(business),len(pic),len(person_result),len(pic_bus))

150346 200098 200098 199994


In [ ]:
# # pic number 数据量 199994
df1 = pd.merge(pic_bus[["photo_id","business_id","caption","label","memory_score"]],pic,
               left_on = "photo_id",right_on="pic_filename")
df2 = pd.merge(df1,person_result,on="photo_id")
df3 = pd.merge(df2,business,how='left',on="business_id")
df3.loc[df3['caption'].isnull(),'caption'] = 0
df3.loc[~df3['caption'].isnull(),'caption'] = 1
df3 = protect_excel_formula(df3)
df3.to_excel("data/pic/pic_feature.xlsx",index=False)

199994 Index(['photo_id', 'business_id', 'caption', 'label', 'memory_score',
       'pic_filename', 'average_hue', 'average_saturation', 'average_value',
       'number_face', 'smiling_faces_count', 'sharpness_measure',
       'detected_text', 'uniqueness_score', 'person_count', 'person_exist',
       'objects_content', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')


In [13]:
print(len(df3),df3.columns)

199994 Index(['photo_id', 'business_id', 'caption', 'label', 'memory_score',
       'pic_filename', 'average_hue', 'average_saturation', 'average_value',
       'number_face', 'smiling_faces_count', 'sharpness_measure',
       'detected_text', 'uniqueness_score', 'person_count', 'person_exist',
       'objects_content', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
